In [1]:
import gym, numpy as np
env=gym.make("MountainCar-v0")
env.reset()

array([-0.44716676,  0.        ])

In [2]:
lr = 0.2
discount=0.95
epochs = 25000

In [3]:
def discretize_state(state):
    return tuple(((state-env.observation_space.low) // d_step).astype(np.int))
# discretize_state(env.reset()), q_table[discretize_state(env.reset())]

In [4]:
env.observation_space.high, env.observation_space.low, env.action_space.n

(array([0.6 , 0.07], dtype=float32), array([-1.2 , -0.07], dtype=float32), 3)

In [5]:
states_dim = [20] * len(env.observation_space.high)
d_step = (env.observation_space.high - env.observation_space.low) /states_dim
d_step

array([0.09 , 0.007])

In [6]:
q_table = np.random.uniform(low=-2, high=0, size=states_dim+[env.action_space.n])
q_table.shape

(20, 20, 3)

In [7]:
display_interval = 100
for epoch in range(epochs):
    print(f'Epoch: {epoch}')
    render = not(epoch%display_interval)
    done=False
    discrete_state = discretize_state(env.reset())
    while not done:
        action=np.argmax(q_table[discrete_state])
        new_state, reward, done, _ = env.step(action)
        new_discrete_state = discretize_state(new_state)
        if render:
            env.render()
        if not done:
            max_future_q = np.max(q_table[new_discrete_state])
            cur_q = q_table[discrete_state + (action, )]
            new_q = (1-lr)*cur_q + lr*(reward + discount*max_future_q)

            q_table[discrete_state + (action, )] = new_q

        elif new_state[0] >= env.goal_position:
            q_table[discrete_state + (action, )] = new_q
            print(f'Reached goal in {epoch} epoch')

        discrete_state=new_discrete_state
        
env.close()

KeyboardInterrupt: 